# Run Miocene Simulation on `f19_g16`

In [2]:
%load_ext autoreload
%autoreload 2

import os
import c4p
print(c4p.__version__)
!hostname

2023.8.7
derecho2


In [3]:
case = c4p.CESMCase(
    account='UBRN0018',
    casename='b.e13.B1850C5.f19_g16.icesm13_ihesp.Miocene.001',
    codebase='/glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_lowres_fzhu',
    res='f19_g16',
    compset='1850_CAM50_CLM40%SP_CICE_POP2_RTM_SGLC_SWAV',
    case_root='/glade/work/fengzhu/CESM_cases',
    output_root='/glade/derecho/scratch/fengzhu/CESM_output',
)

>>> CESMCase.account: UBRN0018
>>> CESMCase.casename: b.e13.B1850C5.f19_g16.icesm13_ihesp.Miocene.001
>>> CESMCase.res: f19_g16
>>> CESMCase.machine: derecho
>>> CESMCase.compset: 1850_CAM50_CLM40%SP_CICE_POP2_RTM_SGLC_SWAV
>>> CESMCase.codebase: /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_lowres_fzhu
>>> CESMCase.case_dirpath: /glade/work/fengzhu/CESM_cases/b.e13.B1850C5.f19_g16.icesm13_ihesp.Miocene.001
>>> CESMCase.output_root: /glade/derecho/scratch/fengzhu/CESM_output
>>> CESMCase.output_dirpath: /glade/derecho/scratch/fengzhu/CESM_output/b.e13.B1850C5.f19_g16.icesm13_ihesp.Miocene.001


In [4]:
case.create(run_unsupported=True)

CMD >>> /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_lowres_fzhu/cime/scripts/create_newcase --case /glade/work/fengzhu/CESM_cases/b.e13.B1850C5.f19_g16.icesm13_ihesp.Miocene.001 --res f19_g16 --compset 1850_CAM50_CLM40%SP_CICE_POP2_RTM_SGLC_SWAV --output-root /glade/derecho/scratch/fengzhu/CESM_output --run-unsupported
Compset longname is 1850_CAM50_CLM40%SP_CICE_POP2_RTM_SGLC_SWAV
Compset specification file is /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_lowres_fzhu/cime_config/config_compsets.xml
Compset forcing is 1850
ATM component is CAM cam5 physics:
LND component is clm4.0:Satellite phenology:
ICE component is Sea ICE (cice) model version 4
OCN component is POP2 
ROF component is RTM: River Transport Model
GLC component is Stub glacier (land ice) component
WAV component is Stub wave component
ESP component is 
Pes     specification file is /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_lowres_fzhu/cime_config/config_pes.xml
Compset specific settings: name is FLDS_WISO and value is TRUE
Co

In [5]:
case.xmlquery('RUN_TYPE')

CMD >>> ./xmlquery -p RUN_TYPE
	RUN_TYPE: hybrid


In [6]:
case.xmlchange({
    'RUN_TYPE': 'startup', 'GET_REFCASE': 'FALSE',
    'CAM_CONFIG_OPTS': '"-water_tracer h2o_h216o_hdo_h218o" --append',
    'OCN_TRACER_MODULES': '"iage wiso"',
    'FLDS_WISO': 'TRUE',
    'CLM_FORCE_COLDSTART': 'on',
    'CLM_BLDNML_OPTS': '"-ignore_warnings" --append',
    'OCN_NCPL': '1', 'CPL_SEQ_OPTION': 'CESM1_MOD', # daily coupling
})

CMD >>> ./xmlchange RUN_TYPE=startup
CMD >>> ./xmlchange GET_REFCASE=FALSE
CMD >>> ./xmlchange CAM_CONFIG_OPTS="-water_tracer h2o_h216o_hdo_h218o" --append
CMD >>> ./xmlchange OCN_TRACER_MODULES="iage wiso"
CMD >>> ./xmlchange FLDS_WISO=TRUE
CMD >>> ./xmlchange CLM_FORCE_COLDSTART=on
CMD >>> ./xmlchange CLM_BLDNML_OPTS="-ignore_warnings" --append
CMD >>> ./xmlchange OCN_NCPL=1
CMD >>> ./xmlchange CPL_SEQ_OPTION=CESM1_MOD


In [7]:
ntasks_atm = 468
ntasks_ocn = 108
ntasks_lnd = 144
ntasks_ice = 324
case.xmlchange({
    'NTASKS_CPL': ntasks_atm, 'NTHRDS_CPL': 3, 'ROOTPE_CPL': 0,
    'NTASKS_ATM': ntasks_atm, 'NTHRDS_ATM': 3, 'ROOTPE_ATM': 0,
    'NTASKS_LND': ntasks_lnd, 'NTHRDS_LND': 1, 'ROOTPE_LND': 0,
    'NTASKS_ICE': ntasks_ice, 'NTHRDS_ICE': 1, 'ROOTPE_ICE': ntasks_lnd,
    'NTASKS_OCN': ntasks_ocn, 'NTHRDS_OCN': 3, 'ROOTPE_OCN': ntasks_atm,
    'NTASKS_ROF': ntasks_lnd, 'NTHRDS_ROF': 1, 'ROOTPE_ROF': 0,
})

CMD >>> ./xmlchange NTASKS_CPL=468
CMD >>> ./xmlchange NTHRDS_CPL=3
CMD >>> ./xmlchange ROOTPE_CPL=0
CMD >>> ./xmlchange NTASKS_ATM=468
CMD >>> ./xmlchange NTHRDS_ATM=3
CMD >>> ./xmlchange ROOTPE_ATM=0
CMD >>> ./xmlchange NTASKS_LND=144
CMD >>> ./xmlchange NTHRDS_LND=1
CMD >>> ./xmlchange ROOTPE_LND=0
CMD >>> ./xmlchange NTASKS_ICE=324
CMD >>> ./xmlchange NTHRDS_ICE=1
CMD >>> ./xmlchange ROOTPE_ICE=144
CMD >>> ./xmlchange NTASKS_OCN=108
CMD >>> ./xmlchange NTHRDS_OCN=3
CMD >>> ./xmlchange ROOTPE_OCN=468
CMD >>> ./xmlchange NTASKS_ROF=144
CMD >>> ./xmlchange NTHRDS_ROF=1
CMD >>> ./xmlchange ROOTPE_ROF=0


In [8]:
mapping_dirpath = '/glade/work/fengzhu/Projects/c4p_gen_mapping_domain'
rof_dirc_dirpath = '/glade/work/fengzhu/Projects/c4p_gen_runoff_direction'
case.xmlchange({
    'ATM_DOMAIN_PATH': mapping_dirpath,
    'LND_DOMAIN_PATH': mapping_dirpath,
    'OCN_DOMAIN_PATH': mapping_dirpath,
    'ICE_DOMAIN_PATH': mapping_dirpath,
    'ATM_DOMAIN_FILE': 'domain.lnd.ne16np4_gx1vMIO.230716.nc',
    'LND_DOMAIN_FILE': 'domain.lnd.ne16np4_gx1vMIO.230716.nc',
    'OCN_DOMAIN_FILE': 'domain.ocn.gx1vMIO.230716.nc',
    'ICE_DOMAIN_FILE': 'domain.ocn.gx1vMIO.230716.nc',
    'ATM2OCN_FMAPNAME': os.path.join(mapping_dirpath, 'map_ne16np4_TO_gx1vMIO_aave.230716.nc'),
    'ATM2OCN_SMAPNAME': os.path.join(mapping_dirpath, 'map_ne16np4_TO_gx1vMIO_blin.230716.nc'),
    'ATM2OCN_VMAPNAME': os.path.join(mapping_dirpath, 'map_ne16np4_TO_gx1vMIO_patc.230716.nc'),
    'OCN2ATM_FMAPNAME': os.path.join(mapping_dirpath, 'map_gx1vMIO_TO_ne16np4_aave.230716.nc'),
    'OCN2ATM_SMAPNAME': os.path.join(mapping_dirpath, 'map_gx1vMIO_TO_ne16np4_blin.230716.nc'),
    'LND2ROF_FMAPNAME': os.path.join(mapping_dirpath, 'map_ne16np4_TO_r1_nomask_aave.230716.nc'),
    'ROF2LND_FMAPNAME': os.path.join(mapping_dirpath, 'map_r1_nomask_TO_ne16np4_aave.230716.nc'),
    'ROF2OCN_FMAPNAME': os.path.join(mapping_dirpath, 'map_r1_nomask_TO_gx1vMIO_aave.230716.nc'),
    'ROF2OCN_RMAPNAME': os.path.join(rof_dirc_dirpath, 'map_r1_nomask_to_gx1vMIO_nnsm_e1000r300.230719.nc'),
})

CMD >>> ./xmlchange ATM_DOMAIN_PATH=/glade/work/fengzhu/Projects/c4p_gen_mapping_domain
CMD >>> ./xmlchange LND_DOMAIN_PATH=/glade/work/fengzhu/Projects/c4p_gen_mapping_domain
CMD >>> ./xmlchange OCN_DOMAIN_PATH=/glade/work/fengzhu/Projects/c4p_gen_mapping_domain
CMD >>> ./xmlchange ICE_DOMAIN_PATH=/glade/work/fengzhu/Projects/c4p_gen_mapping_domain
CMD >>> ./xmlchange ATM_DOMAIN_FILE=domain.lnd.ne16np4_gx1vMIO.230716.nc
CMD >>> ./xmlchange LND_DOMAIN_FILE=domain.lnd.ne16np4_gx1vMIO.230716.nc
CMD >>> ./xmlchange OCN_DOMAIN_FILE=domain.ocn.gx1vMIO.230716.nc
CMD >>> ./xmlchange ICE_DOMAIN_FILE=domain.ocn.gx1vMIO.230716.nc
CMD >>> ./xmlchange ATM2OCN_FMAPNAME=/glade/work/fengzhu/Projects/c4p_gen_mapping_domain/map_ne16np4_TO_gx1vMIO_aave.230716.nc
CMD >>> ./xmlchange ATM2OCN_SMAPNAME=/glade/work/fengzhu/Projects/c4p_gen_mapping_domain/map_ne16np4_TO_gx1vMIO_blin.230716.nc
CMD >>> ./xmlchange ATM2OCN_VMAPNAME=/glade/work/fengzhu/Projects/c4p_gen_mapping_domain/map_ne16np4_TO_gx1vMIO_patc.2

In [9]:
case.setup()

CMD >>> ./case.setup
copying /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_lowres_fzhu/cime/config/cesm/machines/Depends.intel -> /glade/work/fengzhu/CESM_cases/b.e13.B1850C5.f19_g16.icesm13_ihesp.Miocene.001
job is case.run USER_REQUESTED_WALLTIME None USER_REQUESTED_QUEUE None
Creating batch scripts
Writing case.run script from input template /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_lowres_fzhu/cime/config/cesm/machines/template.case.run
Creating file .case.run
Writing case.st_archive script from input template /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_lowres_fzhu/cime/config/cesm/machines/template.st_archive
Creating file case.st_archive
copying /glade/work/fengzhu/CESM_cases/b.e13.B1850C5.f19_g16.icesm13_ihesp.Miocene.001/env_mach_pes.xml -> /glade/work/fengzhu/CESM_cases/b.e13.B1850C5.f19_g16.icesm13_ihesp.Miocene.001/LockedFiles
copying /glade/work/fengzhu/CESM_cases/b.e13.B1850C5.f19_g16.icesm13_ihesp.Miocene.001/env_batch.xml -> /glade/work/fengzhu/CESM_cases/b.e13.B1850C5.f19_g16

/glade/work/fengzhu/CESM_cases/b.e13.B1850C5.f19_g16.icesm13_ihesp.Miocene.001/env_mach_specific.xml already exists, delete to replace


If an old case build already exists, might want to run 'case.build --clean' before building
You can now run './preview_run' to get more info on how your case will be run


In [19]:
case.write_file('user_nl_cam', '''
bnd_topo    = '/glade/u/home/fengzhu/CESM/tutorials/pacosta/cami_bnd_topo_0096x0144_MIO_c130424.nc'

co2vmr      = 1708.2e-6
ch4vmr      = 597.0e-9
n2ovmr      = 262.0e-9
f11vmr      = 0.0
f12vmr      = 0.0

cldfrc_rhminl  = 0.8870D0
micro_mg_dcs   = 525.D-6
dust_emis_fact = 0.95D0

! do not apply the lower-bounnd limiter on isotopic water vapor
! because PHIS in the model is always smaller than this crit value
wtrc_limiter_phis_crit = 100000.0

!! high-frequency output turned off for LR runs
''')

In [11]:
case.write_file('user_nl_pop', '''
! namelists for wiso
wiso_use_nml_surf_vals      = .false.
wiso_comp_surf_avg_freq_opt = 'nmonth'
wiso_comp_surf_avg_freq     = 1

overflows_on            = .false.
overflows_interactive   = .false.
horiz_grid_file         = '/glade/u/home/fengzhu/CESM/tutorials/pacosta/grid.gx1vMIO.pop.da'
topography_file         = '/glade/u/home/fengzhu/CESM/tutorials/pacosta/gridkmt.gx1vMIO.202.182.0.5.40.50_pop.da'
region_mask_file        = '/glade/u/home/fengzhu/CESM/tutorials/pacosta/makeocn/region.ieeei4'
lat_aux_begin           = -90.0
lat_aux_end             = 90.0
lat_aux_grid_type       = 'user-specified'
n_lat_aux_grid          = 90
moc_requested           = .true.
n_heat_trans_requested  = .true.
n_salt_trans_requested  = .true.
n_transport_reg         = 1
transport_reg2_names    = 'Eq Bay','Mid-ocean Ridge'
dt_count                = 23
time_mix_freq           = 17
lhoriz_varying_bckgrnd  = .false.
ldiag_velocity          = .false.
ltidal_mixing           = .false.
bckgrnd_vdc1            = 0.524
bckgrnd_vdc2            = 0.313
sw_absorption_type      = 'jerlov'
''')

In [21]:
case.write_file('user_nl_clm', '''
urban_hac   = 'OFF'
fsurdat     = '/glade/u/home/fengzhu/CESM/tutorials/pacosta/surfdata_1.9x2.5_mio_210507.nc'
''')

In [13]:
case.preview_namelists()

CMD >>> ./preview_namelists
Creating component namelists
  2023-08-14 17:54:06 atm
   Calling /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_lowres_fzhu/components/cam//cime_config/buildnml
     ...calling cam buildcpp to set build time options
CAM namelist copy: file1 /glade/work/fengzhu/CESM_cases/b.e13.B1850C5.f19_g16.icesm13_ihesp.Miocene.001/Buildconf/camconf/atm_in file2 /glade/derecho/scratch/fengzhu/CESM_output/b.e13.B1850C5.f19_g16.icesm13_ihesp.Miocene.001/run/atm_in 
  2023-08-14 17:54:06 lnd
   Calling /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_lowres_fzhu/components/clm//cime_config/buildnml
     ...calling clm buildcpp to set build time options


  2023-08-14 17:54:06 ice
   Calling /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_lowres_fzhu/components/cice//cime_config/buildnml
     ...buildnml calling cice buildcpp to set build time options
  2023-08-14 17:54:07 ocn
   Calling /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_lowres_fzhu/components/pop//cime_config/buildnml
     ... buildnml: calling pop buildcpp to set build time options
     POP build-namelist: ocn_grid is gx1v6 
POP build-namelist: ocn_tracer_modules are iage wiso
  2023-08-14 17:54:19 rof
   Calling /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_lowres_fzhu/components/rtm//cime_config/buildnml
  2023-08-14 17:54:19 glc
   Calling /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_lowres_fzhu/cime/src/components/stub_comps/sglc/cime_config/buildnml
  2023-08-14 17:54:19 wav
   Calling /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_lowres_fzhu/cime/src/components/stub_comps/swav/cime_config/buildnml
  2023-08-14 17:54:19 esp
   Calling /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_lowres_fzhu/

In [14]:
case.build()

CMD >>> qcmd -N test -q main -l select=1:ncpus=36:mpiprocs=36:mem=64GB -l walltime=06:00:00 -A UBRN0018 -- ./case.build
Waiting on job launch; 1258398.desched1 with qsub arguments:
    qsub  -l select=1:ncpus=36:mpiprocs=36:mem=64GB -A UBRN0018 -N test -q main@desched1 -l walltime=06:00:00

Building case in directory /glade/work/fengzhu/CESM_cases/b.e13.B1850C5.f19_g16.icesm13_ihesp.Miocene.001
sharedlib_only is False
model_only is False
Generating component namelists as part of build
Creating component namelists
2023-08-14 17:54:26 atm
Calling /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_lowres_fzhu/components/cam//cime_config/buildnml
...calling cam buildcpp to set build time options
CAM namelist copy: file1 /glade/work/fengzhu/CESM_cases/b.e13.B1850C5.f19_g16.icesm13_ihesp.Miocene.001/Buildconf/camconf/atm_in file2 /glade/derecho/scratch/fengzhu/CESM_output/b.e13.B1850C5.f19_g16.icesm13_ihesp.Miocene.001/run/atm_in
2023-08-14 17:54:26 lnd
Calling /glade/u/home/fengzhu/CESM/iCESM1.3_iHE

In [15]:
case.xmlchange({
    'STOP_OPTION': 'nmonths',
    'STOP_N': '1',
    'REST_N': '1',
    'RESUBMIT': '0',
    'JOB_QUEUE': 'main',
    'JOB_WALLCLOCK_TIME': '12:00:00',
})

CMD >>> ./xmlchange STOP_OPTION=nmonths
CMD >>> ./xmlchange STOP_N=1
CMD >>> ./xmlchange REST_N=1
CMD >>> ./xmlchange RESUBMIT=0
CMD >>> ./xmlchange JOB_QUEUE=main
CMD >>> ./xmlchange JOB_WALLCLOCK_TIME=12:00:00


In [22]:
case.submit()

CMD >>> ./case.submit
copying /glade/work/fengzhu/CESM_cases/b.e13.B1850C5.f19_g16.icesm13_ihesp.Miocene.001/env_batch.xml -> /glade/work/fengzhu/CESM_cases/b.e13.B1850C5.f19_g16.icesm13_ihesp.Miocene.001/LockedFiles
copying /glade/work/fengzhu/CESM_cases/b.e13.B1850C5.f19_g16.icesm13_ihesp.Miocene.001/env_batch.xml -> /glade/work/fengzhu/CESM_cases/b.e13.B1850C5.f19_g16.icesm13_ihesp.Miocene.001/LockedFiles
Creating component namelists
  2023-08-14 18:08:53 atm
   Calling /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_lowres_fzhu/components/cam//cime_config/buildnml
CAM namelist copy: file1 /glade/work/fengzhu/CESM_cases/b.e13.B1850C5.f19_g16.icesm13_ihesp.Miocene.001/Buildconf/camconf/atm_in file2 /glade/derecho/scratch/fengzhu/CESM_output/b.e13.B1850C5.f19_g16.icesm13_ihesp.Miocene.001/run/atm_in 
  2023-08-14 18:08:54 lnd
   Calling /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_lowres_fzhu/components/clm//cime_config/buildnml


  2023-08-14 18:08:54 ice
   Calling /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_lowres_fzhu/components/cice//cime_config/buildnml
  2023-08-14 18:08:54 ocn
   Calling /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_lowres_fzhu/components/pop//cime_config/buildnml
     POP build-namelist: ocn_grid is gx1v6 
POP build-namelist: ocn_tracer_modules are iage wiso
  2023-08-14 18:09:06 rof
   Calling /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_lowres_fzhu/components/rtm//cime_config/buildnml
  2023-08-14 18:09:07 glc
   Calling /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_lowres_fzhu/cime/src/components/stub_comps/sglc/cime_config/buildnml
  2023-08-14 18:09:07 wav
   Calling /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_lowres_fzhu/cime/src/components/stub_comps/swav/cime_config/buildnml
  2023-08-14 18:09:07 esp
   Calling /glade/u/home/fengzhu/CESM/iCESM1.3_iHESP_lowres_fzhu/cime/src/components/stub_comps/sesp/cime_config/buildnml
  2023-08-14 18:09:07 cpl
   Calling /glade/u/home/fengzhu/CESM/iCESM1.3_i

Model clm no file specified for finidat
submit_jobs case.run
Submit job case.run
Submit job case.st_archive


Submitted job id is 1258408.desched1
Submitted job case.run with id 1258407.desched1
Submitted job case.st_archive with id 1258408.desched1
